## Millikan Oil Drop Experiment

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
import math
import pymc3 as pm
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

Assume that we have made the following measurements of the charge on the oil drops. These are all in units of 1e-19 coulombs.

In [ ]:
measurements = np.array([1.26, 2.88, 4.86, 3.66, 5.48, 1.02, 3.00, 3.26, 4.87, 1.60])

We can build a probablistic model to describe this noisy measurement process.

In [ ]:
N = len(measurements)

model = pm.Model()

with model:
    
    charge = pm.Exponential("charge", 1.0)

    number = pm.Poisson('number', 1.0, shape=(N))
        
    mu = (1 + number) * charge
    
    sd = pm.Exponential("sd", 10.0)

    observation = pm.Normal("obs", mu=mu, sd=sd, observed=measurements) 

We run the model with some default settings.

In [ ]:
with model:
    
    trace = pm.sample(2000)

We can quickly review the trace. In this case we need to select the random variables because *traceplot* does not know how to draw the discrete *number* random variable.

In [ ]:
pm.traceplot(trace, var_names=["charge", "sd"]);

Finally when we are happy we can generate some final plots and report on the results.

In [ ]:
fig, axes = plt.subplots(2, 1, figsize=(6, 6))

axes[0].hist(trace['charge'], histtype='stepfilled', bins=100, alpha=0.85, color="#467821", density=True)
axes[0].set_xlim(1, 2)
axes[0].set_title('Charge On The Electron ($e$)')
axes[0].set_xlabel('Coulombs (x $10^{-19}$ C)')

axes[1].hist(trace['sd'], histtype='stepfilled', bins=100, alpha=0.85, color="#A60628", density=True)
axes[1].set_xlim(0, 1)
axes[1].set_title('Measurement Error ($\sigma$)')
axes[1].set_xlabel('Coulombs (x $10^{-19}$ C)')

plt.tight_layout()

plt.savefig('results.pdf')

We can also extract the values of various prior probabilities and measurements.

In [ ]:
mu = np.mean(trace["charge"])
sd = np.std(trace["charge"])
             
print("Charge on the electron is " + str(np.round(mu, 2)) + ' +/- ' + str(np.round(sd, 2)))

In [ ]:
print(np.round(np.mean( 1 + trace['number'], 0), 3))